In [ ]:
%pip install --upgrade pip

%pip install numpy
%pip install pandas
%pip install matplotlib

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

0         A706918
1         A724273
2         A857105
3         A743852
4         A635072
           ...   
111152    A912346
111153    A912468
111154    A912501
111155    A904754
111156    A912037
Name: id, Length: 111157, dtype: object

In [ ]:
## Data Preprocessing ##


records with missing names: 31383
records that are blank or has numbers: 33168


In [ ]:
# Rebecca's slay data exploration block /ᐠ - ˕ -マ 
# intake time, color, outcome time, date of birth

In [ ]:
# Nneoma's data visualation
print('records with missing names:', df_train['name'].isna().sum())

print('records that are blank or has numbers:', non_alpha_values.sum()) # does not wrok but you get the idea. fix pls